In [0]:
pip install haversine

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName("Datasetexploration").getOrCreate()

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [0]:
#df = spark.read.table("dataset_2017_fordgobike_tripdata_1_csv")

df = spark.read.option("header","true").csv("/FileStore/Dataset_2017_fordgobike_tripdata.csv")
#df = spark.read.csv("/home/julian/FORD_GO_BIKE_PROJECT/Dataset-2017-fordgobike-tripdata.csv",header=True)

In [0]:
df.show(3)

+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+
|start_time|end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id|user_type|member_birth_year|member_gender|     pyment|
+----------+--------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+---------+-----------------+-------------+-----------+
|   57:39.7| 12:50.2|              74|Laguna St at Haye...|           37.77643482|            -122.426244|            43|San Francisco Pub...|          37.7787677|         -122.4159292|     96| Customer|             1987|         Male|credit card|
|   56:3

In [0]:
df.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [0]:
df.count()

Out[11]: 519700

In [0]:
# import packages 
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types


def getNullNanBlankCount(df: DataFrame) -> DataFrame: 
  
  # get all the column names into a list
  allColsList = df.columns
  
  # create an empty list to temporarily store column names and count values
  listBuffer = []
  
  for field in allColsList:   
    # count nulls
    nullCondition = F.isnull(F.col(field))
    nullCount = df.select(F.col(field)).filter(nullCondition).count()  
    
    # count nans
    nanCondition = F.isnan(F.col(field))
    nanCount = df.select(F.col(field)).filter(nanCondition).count()  
    
    # count blank/empty values
    blankCount = df.select(F.col(field)).filter(F.col(field) == " ").count()  
     
    # append column names and count values to the list
    listBuffer.append((field, nullCount, nanCount, blankCount))

  # convert listBuffer into a new dataframe
  resDf = spark.createDataFrame(listBuffer, ["table_column_name", "null_count", "nan_count", "blank_count"])
 
  return resDf
  
  

In [0]:
df.transform(getNullNanBlankCount).show(truncate=False)

+-----------------------+----------+---------+-----------+
|table_column_name      |null_count|nan_count|blank_count|
+-----------------------+----------+---------+-----------+
|start_time             |0         |0        |0          |
|end_time               |0         |0        |0          |
|start_station_id       |0         |0        |0          |
|start_station_name     |0         |0        |0          |
|start_station_latitude |0         |0        |0          |
|start_station_longitude|0         |0        |0          |
|end_station_id         |0         |0        |0          |
|end_station_name       |0         |0        |0          |
|end_station_latitude   |0         |0        |0          |
|end_station_longitude  |0         |0        |0          |
|bike_id                |0         |0        |0          |
|user_type              |0         |0        |0          |
|member_birth_year      |66541     |0        |0          |
|member_gender          |66462     |0        |0         

We can easily notice that there are only 2 columns with nan values and we will see how to deal with them later on

In [0]:
df.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [0]:
#df1 = copy.deepcopy(X.schema)

In [0]:
df1 = df

In [0]:
df1.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [0]:
df1.filter(df1['member_birth_year'].isNull()).count()

Out[7]: 66541

## 1. Looking for a way of getting the distance travelled for each trip

In [0]:

df1 = df1.withColumn('start_station_latitude',df['start_station_latitude'].cast("double")).\
         withColumn('start_station_longitude',df['start_station_longitude'].cast("double")).\
        withColumn('end_station_latitude',df['end_station_latitude'].cast("double")).\
        withColumn('end_station_longitude',df['end_station_longitude'].cast("double"))

In [0]:
df1.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import haversine
from haversine import haversine, Unit


In [0]:
from pyspark.sql.functions import udf , round
from pyspark.sql.types import StringType,DoubleType
# defining the function you want 
def distance(lat1,long1,lat2,long2):
    loc1 = (lat1,long1)
    loc2 = (lat2,long2)
    distance1 =haversine(loc1,loc2, Unit.KILOMETERS)
    return distance1

distanceUDF = udf(distance , DoubleType())

In [0]:
df1 = df1.withColumn("distance_travelled", distanceUDF(df1['start_station_latitude'],df1['start_station_longitude'],df1['end_station_latitude'] ,df1['end_station_longitude']))#.select('distance_travelled').show()
                                #.withColumn('distance_travelled',round(df1['distance_travelled'],2)).select('distance_travelled').show()                                                            
                        

In [0]:
df1.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)



In [0]:
df1 = df1.withColumn('distance_travelled',round(df1['distance_travelled'],2))#.select('distance_travelled').show() 

In [0]:
df1.select('distance_travelled').show() 

+------------------+
|distance_travelled|
+------------------+
|              0.94|
|              3.07|
|               0.0|
|              1.05|
|              0.64|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|              0.77|
|              0.77|
|              1.52|
|              1.52|
|              1.42|
|              1.05|
|              1.05|
|              2.86|
|              2.86|
|              2.86|
+------------------+
only showing top 20 rows



## 2. let us now see if we can get the time taken by each trip

#### Let us first find a way of organising the start and end time column to a proper format

In [0]:
df1.select('start_time','end_time').printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)



In [0]:
from pyspark.sql.functions import udf
def attachinghour(mystring):
    onehour = '01:'
    good_format = onehour+mystring
    return good_format
attachinghourUDF = udf(lambda x : attachinghour(x))

In [0]:
df1 = df1.withColumn('start_time', attachinghourUDF(df1['start_time'])).withColumn('end_time', attachinghourUDF(df1['end_time']))#.select('start_time','end_time').show(5)

In [0]:
a = '01:57:39.7'
b = '01:12:50.2'
len(a)

Out[48]: 10

In [0]:
#a[1]
if a[3:5] > b[3:5]:
    print('the hour should be shifted')
    a = a.replace(a[0:2],'02')
    print(a)
else:
    print('Nothing should be done')

the hour should be shifted
02:57:39.7


In [0]:
from pyspark.sql.types import StringType,DoubleType
# defining the function you want 
def moving_up_thehour(starttime,endtime):
    if starttime[3:5] > endtime[3:5]:
        return endtime.replace(endtime[0:2],'02')
    else:
        return endtime

moving_hour_UDF = udf(moving_up_thehour , StringType())

In [0]:
df1 = df1.withColumn("end_time", moving_hour_UDF(df1['start_time'],df1['end_time']))#.select('start_time','end_time').show(10)

In [0]:
df1.select('start_time','end_time').show(10)

+----------+----------+
|start_time|  end_time|
+----------+----------+
|01:57:39.7|02:12:50.2|
|01:56:34.8|02:49:55.6|
|01:45:48.4|02:28:36.9|
|01:31:10.6|01:47:23.5|
|01:23:14.0|01:29:57.6|
|01:51:00.9|02:24:47.2|
|01:49:28.4|02:04:35.6|
|01:46:37.2|01:58:51.2|
|01:37:07.5|01:46:18.3|
|01:35:38.1|01:46:17.1|
+----------+----------+
only showing top 10 rows



In [0]:
df1

In [0]:
from pyspark.sql.functions import *
df1 = df1.withColumn('start_time', to_timestamp(col('start_time'),"HH:mm:ss.SSS"))\
    .withColumn('end_time', to_timestamp(col('end_time'),"HH:mm:ss.SSS"))

In [0]:
df1.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)



In [0]:
df1.withColumn('trip_time', df1['start_time'] - df1['end_time']).select('trip_time').show(truncate=False)
# From the obtained results this method will not probably work

+--------------------------------------+
|trip_time                             |
+--------------------------------------+
|INTERVAL '-0 00:15:10.5' DAY TO SECOND|
|INTERVAL '-0 00:53:20.8' DAY TO SECOND|
|INTERVAL '-0 00:42:48.5' DAY TO SECOND|
|INTERVAL '-0 00:16:12.9' DAY TO SECOND|
|INTERVAL '-0 00:06:43.6' DAY TO SECOND|
|INTERVAL '-0 00:33:46.3' DAY TO SECOND|
|INTERVAL '-0 00:15:07.2' DAY TO SECOND|
|INTERVAL '-0 00:12:14' DAY TO SECOND  |
|INTERVAL '-0 00:09:10.8' DAY TO SECOND|
|INTERVAL '-0 00:10:39' DAY TO SECOND  |
|INTERVAL '-0 00:54:52.2' DAY TO SECOND|
|INTERVAL '-0 00:52:57.8' DAY TO SECOND|
|INTERVAL '-0 00:36:23.1' DAY TO SECOND|
|INTERVAL '-0 00:36:11' DAY TO SECOND  |
|INTERVAL '-0 00:44:57.6' DAY TO SECOND|
|INTERVAL '-0 00:25:44.1' DAY TO SECOND|
|INTERVAL '-0 00:24:34.3' DAY TO SECOND|
|INTERVAL '-0 00:23:17' DAY TO SECOND  |
|INTERVAL '-0 00:25:32.5' DAY TO SECOND|
|INTERVAL '-0 00:20:16.1' DAY TO SECOND|
+--------------------------------------+
only showing top

In [0]:
#Another approach to get the duration in minutes
df1 = df1.withColumn('trip_duration_seconds',col("end_time").cast("long") - col('start_time').cast("long"))#.select('trip_duration_seconds').show(10,truncate=False)
# got results obtained

In [0]:
# adding the duration in minutes
df1 = df1.withColumn('trip_duration_minutes',round((col("end_time").cast("long") - col('start_time').cast("long"))/60))#.select('trip_duration_minutes').show(10,truncate=False)


In [0]:
df1.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)
 |-- trip_duration_seconds: long (nullable = true)
 |-- trip_duration_minutes: double (nullable = true)



---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-2961780054634668> in <cell line: 1>()
----> 1 import org.apache.spark.sql.functions

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    169             # Import the desired module. If you’re seeing this while debugging a failed import,
    170             # look at preceding stack frames for relevant error information.
--> 171             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    172 
    173             is_root_import = thread_local._nest_level == 1

ModuleNotFoundError: No module named 'org.apache.spark.sql'

##### Let us try using the SQL syntax

In [0]:
df1.createOrReplaceTempView('df1')

In [0]:
spark.sql('SELECT * FROM df1 ').show()


+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|          start_time|            end_time|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|distance_travelled|
+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+
|1970-01-01 01:57:...|1970-01-01 02:12:...|              74|Laguna St at Haye...|           37.77643482|            -122.426244|  

In [0]:
spark.sql('SELECT  TIMESTAMPDIFF(SECOND, start_time, end_time) AS trip_duration_seconds FROM df1;').select('trip_duration_seconds').show(10)

+---------------------+
|trip_duration_seconds|
+---------------------+
|                  910|
|                 3200|
|                 2568|
|                  972|
|                  403|
|                 2026|
|                  907|
|                  734|
|                  550|
|                  639|
+---------------------+
only showing top 10 rows



In [0]:
New_df.show(1)

+---------------------+
|trip_duration_seconds|
+---------------------+
|   15.166666666666666|
+---------------------+
only showing top 1 row



In [0]:
spark.sql('SELECT TIMESTAMPDIFF(MINUTE, start_time, end_time) AS trip_time_minutes FROM df1;').select('trip_time_minutes').show(10)

+-----------------+
|trip_time_minutes|
+-----------------+
|               15|
|               53|
|               42|
|               16|
|                6|
|               33|
|               15|
|               12|
|                9|
|               10|
+-----------------+
only showing top 10 rows



## 3. Let us now get the cost for each trip by assuming that the cost is 0.35 cent per minute

In [0]:
df1.printSchema()

root
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- distance_travelled: double (nullable = true)
 |-- trip_duration_seconds: long (nullable = true)
 |-- trip_duration_minutes: double (nullable = true)



In [0]:
df1 = df1.withColumn('trip_fee(cents)', round(df1['trip_duration_minutes']*0.35,2))#.select('trip_fee(cents)').show(10)

In [0]:
df1.select('trip_fee(cents)').show(10)

+---------------+
|trip_fee(cents)|
+---------------+
|           5.25|
|          18.55|
|          15.05|
|            5.6|
|           2.45|
|           11.9|
|           5.25|
|            4.2|
|           3.15|
|           3.85|
+---------------+
only showing top 10 rows



Out[44]: str